# Calculating column density from a $^{13}CO$ observation of MAXI J1348-630

In this document we will go through several steps taken to get a final $^{13}CO$ column density map. There are several assumptions that need to be revisited for future use of this document if more data is available. They will be adressed when they appear.

The code was produced by Dr. Alexandra Tetarenko.

In [83]:
from spectral_cube import SpectralCube
from astropy import units as u
import astropy.constants as con
import numpy as np
from astropy.io import fits
import pylab as pl
import math as ma
import matplotlib.pyplot as plt
from astropy.table import hstack
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.axes_grid1.inset_locator import mark_inset,zoomed_inset_axes
import matplotlib.cm as cm
from astropy import wcs
from astropy.coordinates import SkyCoord,Angle
import matplotlib.patches as patches
import matplotlib
import matplotlib.colors as colors
import mpmath
from astropy.table import hstack
import scipy.ndimage as nd
import matplotlib as mpl
from matplotlib.patches import Rectangle
import aplpy
from matplotlib import rc
import os
from astropy.convolution import Gaussian1DKernel
import radio_beam


datadir = '/Users/student/Desktop/localdata/MAXI_J1348/data/' #We first define the data directory.

## Selecting the velocity range:

Given that your observation might feature foreground and background emission outside of a given velocity range, it is important to first get the interesting data out of the full cube.

Also ,we apply a unit conversion when necessary.


In [86]:
def makeVarr(fitsfile):
    header = fits.getdata(fitsfile, header=True)[1]  # We import the fits header.
    X = SpectralCube.read(fitsfile)  # We read the file directly with the spectral cube repository.
    X.allow_huge_operations = True  # This allows operations that might exceed memory limits.

    print(X.unit)  # Check the unit of the FITS file.

    if X.unit == u.K:
        X1 = X  # No conversion needed.
        print('No conversion needed')
    else:
        X1 = X.to(u.K, equivalencies=X.beam.jtok_equiv(X.header['RESTFRQ'] * u.Hz))
        print('Conversion applied')

    Xkms = X1.with_spectral_unit(u.km/u.s, velocity_convention='radio')  # We transform the spectral axis to km/s.
    Xkms2 = Xkms.spectral_slab(vmin * u.km/u.s, vmax * u.km/u.s)  # We take a given width of velocity values.
    Vel = np.array(Xkms2.spectral_axis)  # We extract the subspectral axis with the desired velocities.
    Ts = np.array(Xkms2)  # We now extract the temperatures associated with such velocities.

    return Ts[::-1, :, :], Vel[::-1]  # We return the values in inverse order.

## Making a $T_{max}$ map:

In this step we take our cube slice and create a $T_{max}$ map of the emission. Notice, that to get that value we would normally use our $^{12}CO$ observation of that same transition, which is optically thick, but since we don't have it, we take the approach to use that very same observation.

In [89]:
def make_TMAX(fitsfile,ddir,line,vmin,vmax):
	header = fits.getdata(fitsfile, header=True)[1] #Again, we read the header
	readfile=SpectralCube.read(fitsfile) #We open the fits file with SpectralCube
	readfile.allow_huge_operations=True #We enable operations beyond memory limits
	filekms=readfile.with_spectral_unit(u.km/u.s,velocity_convention='radio') #We read the file with the correct velocity units.
	fileKkms=filekms.to(u.K, equivalencies=filekms.beam.jtok_equiv(filekms.header['RESTFRQ']*u.Hz)) #We transform the emission units to K
	fileKkms2=fileKkms.spectral_slab(vmin*u.km/u.s,vmax*u.km/u.s) #We select a chunk of velocity.
	tmax=fileKkms2.apply_numpy_function(np.nanmax,axis=0) #We select the maximum value of the array for the velocity axis, effectively collapsing the cube over the spectral axis.
	tmax_fix=np.nan_to_num(tmax) #We remove the NaNs and transform them into numbers (0s). 
	fits.writeto(filename=ddir+'T_max'+line+'.fits',output_verify='ignore',overwrite=True,data=tmax_fix,header=header) #We create the Tmax array into a fits file.

## Making a $T_{ex}$ map:

In this step, we take our $T_{max}$ map, and use it to create a $T_{ex}$ map. We will be assuming an optically thick region with an observational formula approach found on Tetarenko et al., 2017:

$T^{obs}_{ex}=\frac{h\nu/k}{\ln \left(1+\frac{h\nu/k}{T_{max}+ h\nu/k C_{CMB}}\right)}, $ for $\tau \rightarrow \infty$,

where $C_{CMB}=\left[\exp \left(\frac{h\nu}{kT_{CMB}}\right)-1\right]^{-1}$.

Noticeably, our $^{13}CO$ observation is not optically thick, hence the validity of this equation is really not quite accurate. However, in lack of a $^{12}CO$ observation for that same transition, this is all that we have. Notice that $T_R$ is our observed temperature at each pixel.

In [92]:
def Tex(nu0,Tmax0):
	"""
    nu0 is reference frequency molecule in Ghz,Tmax0 is reference molecule max T in K
    """
	unitdict={'h':6.626068e-27,'k':1.3806503e-16,'c':2.99792458e10} #We introduce numerical values for Planck's constant, speed of light and Boltzmann's constant.
	h,k,c=unitdict['h'],unitdict['k'],unitdict['c']
	nu=nu0*1e9 #We transform the frequency to Hz
	Tbg=2.73 #Temperature of the CMB background to compute its blackbody emission.
	C=h*nu/k #We define the C value that will appear more than once in our formulas to make them simple.
	Cbg=1./(np.exp((h*nu)/(k*Tbg))-1) #We compute the term associated to the background emission.
	Tex=C/(np.log(1+(C/(Tmax0+C*Cbg)))) #Finally, we derive values for the excitation temp.
	return(Tex) #We return the array with the computed temperatures.

## Estimating optical depth:

At this point we have to produce a $\tau$ map. Normally, this is performed when one can assess more than one observation with different species and get $T_ex$ with the optically thick one. But in this case, again, we only have $^{13}CO$, so we will have to make do with what we have.

We use the same formula found in Tetarenko et al., 2017:


$\tau = -\ln \left( 1 - \frac{kT_R}{h \nu} \left( \frac{1}{\exp\left[\frac{h \nu}{k T_{\text{ex}}}\right] - 1} - \frac{1}{\exp\left[\frac{h \nu}{k T_{\text{bg}}}\right] - 1} \right)^{-1} \right)$

The result of this function is somewhat patchy, with a not so good looking map (a lot of variations), so don't worry if that happens to your data too as long as the peaks of depth are associated to the highest emission.

In [95]:
def Tau(nu1,TV,Tex):
	#nu1 is target frequency molecule in ghz,TV is array of target molecule T in K for a bunch of dV
	unitdict={'h':6.626068e-27,'k':1.3806503e-16,'c':2.99792458e10}
	h,k,c=unitdict['h'],unitdict['k'],unitdict['c']
	nu=nu1*1e9 #Transform the frequency to Hz
	Tbg=2.73 #Define the temperature of the CMB
	C=h*nu/k #Use a simplifying factor for future formulas.
	A=1./(np.exp(C/Tex)-1) #We name the term associated with Tex
	B=1./(np.exp(C/Tbg)-1) #We do the same for that term associated with the CMB temp.
	tau=-np.log(1-((TV/C)*((A-B)**(-1)))) #We finally take the tau full estimation starting from our Tex map.
	return(tau) #tau is in the shape of a spectral subcube for given velocity channels.

## Computing the column density of a given species:

In this bit we calculate the column density of a given level J, which we will later transform into the total column density of the molecule. We take from Wilson et al., 2013 the following expression:


$N_J = \frac{8 \pi \nu^3}{c^3} \frac{(2J+1)}{(J+1)} (1.165 \times 10^{11}) \frac{1}{\mu^2}  \left(1 - e^{-\frac{h \nu}{k T_{\text{ex}}}}\right)^{-1} \int_{v_{min}}^{v_{max}} \tau dv$


where:
- ( $J$) → Rotational quantum number.
- ( $\mu$) → Dipole moment of the molecule (in Debye).
- ( $T_{\text{ex}}$) → Excitation temperature (in K).
- The numeric constant is a combination of the Einstein factor for the given transition ($A_{ul}$) and the population degeneracies for the given levels ($g_u, g_l$). **Ask Alex about this, because the number seems odd in 2017 paper**.


In [98]:
def NJ(nu1,Tex,Tau_array,dV,mu,J):
	#TV is array of target molecule T in K for a bunch of dV (array also in km/s)
	unitdict={'h':6.626068e-27,'k':1.3806503e-16,'c':2.99792458e10}
	h,k,c=unitdict['h'],unitdict['k'],unitdict['c']
	nu=nu1*1e9
	prefac = (8 * np.pi * 1*u.GHz**3 / (con.c**3)) * u.km/u.s * u.s #we compute the prefactor for the final formula assigning the correct units
	C=(prefac.to(u.cm**-2)).value #we transform to cm since our final values will be in cm**{-2}
	col=C*((2*J+1)/(J+1))*(0.858e11)*(mu)**(-2)*(1-np.exp(-h*nu/(k*Tex)))**(-1) #We compute the scaling factor used on tau to get Nj
	inte=np.trapz(np.nan_to_num(Tau_array),dV,axis=0) #We integrade optical depth over the velocity axis using the trapezoidal method.
	fits.writeto(filename=datadir+'column_plots/'+line+'_inte.fits',output_verify='ignore', overwrite=True,data=inte,header=header) #We write a fits file with the N_j
	return(col*inte) #We return the scaled factor, so an N_j matrix.

## Computing the partition function for a give $T_{ex}$ map:

In order to extrapolate the column density of an energy level J to a full molecule we must take into account the partition function for all the different rotational levels. In this case we take for each $T_{ex}$ value in our map:

$$Z (T_{ex}) = \sum_{J=0}^{6}(2J+1)\cdot e^{-\frac{2.65 J(J+1)}{T_{ex}}}$$

Where we take into account the first 6 rotational levels of the $^{13}CO$ molecule.



In [101]:
def Zpart(Tex_array):
	ZZ=np.zeros((Tex_array.shape[0],Tex_array.shape[1]))
	for kk in range(0,(Tex_array.shape[0])):
		for ll in range(0,(Tex_array.shape[1])):
			val=Tex_array[kk,ll]
			ZZ[kk,ll]=mpmath.nsum(lambda x: (2*x+1)*mpmath.exp(-2.65*x*(x+1)/val), [0,6])
	return(ZZ)

## Transforming $N_{J}$ to $N_{tot}$:

In this last part we take our $N_J$ map and transform it into $N_{tot}$ by taking into account our full partition function:

$$N_{tot} = N_J \cdot\frac{Z(T_{ex})}{2J+1}\cdot e^{\frac{2.65 J(J+1)}{T_{ex}}}$$

Where the 2.65 value is the $hB_e/k = 2.65 K$ ratio specific to $CO$.

In [104]:
def Ntot(Tex,Nj,J):
	ZZ=Zpart(Tex)/(2*J+1)
	NN=Nj*ZZ*np.exp(2.65*(J*(J+1.))/Tex)
	return(NN)


## Final production of column density maps:

In [107]:
Tex_vec=np.vectorize(Tex)
Tau_vec=np.vectorize(Tau)



#per spectra version

fitsfile='/Users/student/Desktop/localdata/MAXI_J1348/data/J1348_630_12m+7m_13co10.fits' #We point to the file


line='CO13_comb'

dipolem=0.122#CO
moment=0
flag='K'
vmin=-54 #Minimum velocity of our original observation cube
vmax=-50 #Maximum velocity of our original observation cube
cell=0.5#CO--1.5/0.5

print('Starting...')

make_TMAX(fitsfile,datadir+'column_plots/',line,vmin,vmax)
print('TMAX map done.')
tmax_fits=datadir+'column_plots/'+'T_max'+line+'.fits'
print('TMAX fits done.') 
#We make and save the Tmax map

hdtmax=fits.open(tmax_fits)[0]
dat=np.nan_to_num(hdtmax.data)
#We open our Tmax and correct the NaNs to 0s

freq=110.20135#CO13 1-0 transition frequency
J=0#J+1->J
fr=np.empty((dat.shape[0],dat.shape[1]))
fr.fill(freq) #We create an array with the shape of the T_max map filled with the transition frequency value
tm=np.zeros((dat.shape[0],dat.shape[1]))
fr=np.zeros((dat.shape[0],dat.shape[1]))
#We create empty 0 matrices that we will use in a minute

print('Info input done.')

for kk in range(0,(dat.shape[0])):
	for ll in range(0,(dat.shape[1])):
		val=dat[kk,ll]
		tm[kk,ll]=val
		fr[kk,ll]=freq
        #We use our empty matrices to fill them with frequency and temperature values.
        
Tex_array=Tex_vec(fr,tm)

#We start executing our functions to get to our final Ntot calculation.
print('Tex map done.')
header = fits.getdata(tmax_fits, header=True)[1]
fits.writeto(filename=datadir+'column_plots/'+line+'_tex.fits',output_verify='ignore',\
	overwrite=True,data=Tex_array,header=header)
TV,dV=makeVarr(fitsfile)
print('Tex fits done.')
Tau_array=Tau_vec(fr,TV,Tex_array)
print('Tau map done.')
Tau_array[Tau_array == np.inf] = np.nanmax(Tau_array[Tau_array != np.inf])
fits.writeto(filename=datadir+'column_plots/'+line+'_tau.fits',output_verify='ignore',\
	overwrite=True,data=Tau_array,header=header)
print('Tau fits done.')
NJ_array=NJ(fr,Tex_array,Tau_array,dV,dipolem,J)
fits.writeto(filename=datadir+'column_plots/'+line+'_nj.fits',output_verify='ignore',\
	overwrite=True,data=NJ_array,header=header)
Nfact_array=Ntot(Tex_array,NJ_array,J)
print('NJ fits done.')

colum=Nfact_array
#0.2 for Co
colum[tm<0.2]=0. #We filter our matrix with a hard threshold of 0.2

fits.writeto(filename=datadir+'column_plots/'+line+'_column.fits',output_verify='ignore',\
	overwrite=True,data=colum,header=header)

print('NTOT fits done.')

Starting...
TMAX map done.
TMAX fits done.
Info input done.
Tex map done.
K
No conversion needed


/Users/student/anaconda3/lib/python3.11/site-packages/spectral_cube/spectral_cube.py:439: RuntimeWarning: All-NaN slice encountered
  out = function(self._get_filled_data(fill=fill,
/var/folders/2l/vhjdf14d2vjd4qffvsk5p19r0000gq/T/ipykernel_12879/191859481.py:10: RuntimeWarning: invalid value encountered in log
  tau=-np.log(1-((TV/C)*((A-B)**(-1)))) #We finally take the tau full estimation starting from our Tex map.
/var/folders/2l/vhjdf14d2vjd4qffvsk5p19r0000gq/T/ipykernel_12879/191859481.py:10: RuntimeWarning: divide by zero encountered in log
  tau=-np.log(1-((TV/C)*((A-B)**(-1)))) #We finally take the tau full estimation starting from our Tex map.


Tex fits done.
Tau map done.
Tau fits done.
NJ fits done.
NTOT fits done.
